Les k plus proches voisins (k Nearest Neighbors – kNN)
===

Regarde tous les points, selectionne les k plus proche et analyse leur classe. La majorité l'emporte. Très simple et efficace, surtout sur de petits dataset et avec une bonne valeur de k (ni trop petite (sensible aux aberrations), ni trop grande (perte de précision)).

Avantages
- L’algorithme est super simple et facile à mettre en œuvre.
- Il n’est pas nécessaire de construire un modèle, d’ajuster plusieurs paramètres ou de faire des hypothèses supplémentaires.
- L’algorithme est polyvalent. Il peut être utilisé pour la classification, la régression et la recherche d’informations (comme nous le verrons dans la section suivante).

Inconvénients
- L’algorithme ralentit considérablement à mesure que le nombre d’observations et/ou de variables dépendantes/indépendantes augmente. En effet, l’algorithme parcourt l’ensemble des observations pour calculer chaque distance.

In [12]:
from tools import find_file
from collections import Counter
import math

In [2]:
def mean(labels):
    """Calcul de la moyenne des k étiquettes"""
    return sum(labels) / len(labels)

def mode(labels):
    """Calcul du mode des k étiquettes"""
    return Counter(labels).most_common(1)[0][0]

def euclidean_distance(point1, point2):
    """Calcul de la distance euclidienne"""
    sum_squared_distance = 0
    for i in range(len(point1)):
        sum_squared_distance += math.pow(point1[i] - point2[i], 2)
    return math.sqrt(sum_squared_distance)

In [3]:
# 1. Charger les données
# 2. Initialiser k au nombre de plus proches voisins choisi

def knn(data, query, k, distance_fn, choice_fn):
    """Algorithme des k plus proches voisins kNN"""
    
    neighbor_distances_and_indices = []
    
    # 3. Pour chaque observation des données
    for index, example in enumerate(data):
        # 3.1 Calculer la distance entre notre requête et l'observation itérative actuelle
        # de la boucle depuis les données.
        distance = distance_fn(example[:-1], query)
        
        # 3.2 Ajouter la distance et l'indice de l'observation concernée 
        # à une collection ordonnée de données
        neighbor_distances_and_indices.append((distance, index))
    
    # 4. Trier cette collection ordonnée contenant distances et indices de
    # la plus petite distance à la plus grande (dans l'ordre croissant)
    sorted_neighbor_distances_and_indices = sorted(neighbor_distances_and_indices)
    
    # 5. Sélectionner les k premières entrées de la précédente collection de données
    k_nearest_distances_and_indices = sorted_neighbor_distances_and_indices[:k]
    
    # 6. Obtenir les étiquettes des k entrées sélectionnées
    k_nearest_labels = [data[i][1] for distance, i in k_nearest_distances_and_indices]
    # 7. Si régression (choice_fn = mean), retourner la moyenne des k étiquettes
    # 8. Si classification (choice_fn = mode), retourner le mode des k étiquettes
    return k_nearest_distances_and_indices , choice_fn(k_nearest_labels)

In [9]:
'''
# Données de régression
# 
# Colonne 0: taille (en pouces)
# Colonne 1: poids (en livres)
'''
reg_data = [
   [65.75, 112.99],
   [71.52, 136.49],
   [69.40, 153.03],
   [68.22, 142.34],
   [67.79, 144.30],
   [68.70, 123.30],
   [69.80, 141.49],
   [70.01, 136.46],
   [67.90, 112.37],
   [66.49, 127.45],
]

# Question:
# Compte tenu des données dont nous disposons, quelle est la meilleure 
# estimation du poids d'une personne mesurant 60 pouces?
reg_query = [60]
k = 3
reg_k_nearest_neighbors, reg_prediction = knn(
    reg_data, reg_query, k=k, distance_fn=euclidean_distance, choice_fn=mean
)
print(f"poid estimé d'une personne de 60 pouces (152.4 cm) : {reg_prediction} pounds, soit {reg_prediction/ 2.205} Kg.\nListe des {k} plus proches voisins: ")
reg_k_nearest_neighbors

poid estimé d'une personne de 60 pouces (152.4 cm) : 128.24666666666667 pounds, soit 58.16175359032502 Kg.
Liste des 3 plus proches voisins: 


[(5.75, 0), (6.489999999999995, 9), (7.790000000000006, 4)]

In [11]:
'''
# Données de Classification
# 
# Colonne 0: age
# Colonne 1: aime l'ananas
'''
clf_data = [
   [22, 1],
   [23, 1],
   [21, 1],
   [18, 1],
   [19, 1],
   [25, 0],
   [27, 0],
   [29, 0],
   [31, 0],
   [45, 0],
]
# Question:
# À la lumière des données dont nous disposons, une personne de 33 ans aime-t-elle
# l'ananas sur sa pizza?
clf_query = [33]
clf_k_nearest_neighbors, clf_prediction = knn(
    clf_data, clf_query, k=3, distance_fn=euclidean_distance, choice_fn=mode
)
print(f"Une personne de 33 ans aime-t-elle l'ananas sur sa pizza : {clf_prediction}.\nListe des {k} plus proches voisins: ")
clf_k_nearest_neighbors

Une personne de 33 ans aime-t-elle l'ananas sur sa pizza : 0.
Liste des 3 plus proches voisins: 


[(2.0, 8), (4.0, 7), (6.0, 6)]

Cet algorithme est très intéressant quand on veut faire un système de recommandation, on analyse qui à des gouts similaires pour proposer des articles pertinents (Amazon, Medium, Netflix, Youtube). Bien que ces derniers utilisent des algorithmes plus rapide, on peut imaginer ce système :


*À quelle question essayons-nous de répondre ?*
Compte tenu de notre ensemble de données de films, quels sont les 5 films les plus similaires à une requête de film?

*Recueillir des données de films*
On peut utiliser notre dataset ou un autre que l'on trouve online : https://archive.ics.uci.edu/ml/datasets/Movie

*Exploration, nettoyage et préparation des données*
- valeurs manquantes ? remplacer / renseigner / supprimer ?
- données exclusivement numériques ?

*Utilisation de l'algorithme*
On va d'abords récupérer notre film de référence et mettre ses features dans une liste : **feature vector**. Ensuite on execute kNN. 

In [13]:
def recommend_movies(movie_query, k_recommendations):
    raw_movies_data = []
    
    with open(find_file('movies_recommendation_data.csv'), 'r') as md:
        # Eliminer la première ligne (header)
        next(md)
        # Lire les données
        for line in md.readlines():
            data_row = line.strip().split(',')
            raw_movies_data.append(data_row)
            
    # Préparer les données à être utiliser dans l'algorithme kNN en
    # choissant les colonnes appropriées et en convertissant les colonnes 
    # numériques en nombres car elles ont été lues sous forme de string
    movies_recommendation_data = []
    for row in raw_movies_data:
        data_row = list(map(float, row[2:]))
        movies_recommendation_data.append(data_row)
        
    # Utiliser l'algorithme kNN pour obtenir les 5 films les plus similaires
    # à Pentagon Papers.
    recommendation_indices, _ = knn(
        movies_recommendation_data, movie_query, k=k_recommendations,
        distance_fn=euclidean_distance, choice_fn=lambda x: None
    )
    movie_recommendations = []
    for _, index in recommendation_indices:
        movie_recommendations.append(raw_movies_data[index])
    return movie_recommendations

the_post = [7.2, 1, 1, 0, 0, 0, 0, 1, 0] # feature vector pour Pentagon Papers
recommended_movies = [] # recommend_movies(movie_query=the_post, k_recommendations=5)

# Afficher les titres des films recommandés
for recommendation in recommended_movies:
    print(recommendation[1])